In [1]:
%tensorflow_version 1.x

TensorFlow 1.x selected.


In [2]:
import tensorflow as tf

会话可能拥有的资源，如 `tf.Variable`，`tf.QueueBase` 和 `tf.ReaderBase`。

当这些资源不再需要时，释放这些资源非常重要。

因此，需要调用 `tf.Session` 会话中的 `close()` 方法，或将会话用作上下文管理器。

以下两个例子作用是一样的：

In [9]:
# 1、不使用上下文管理器：
a = tf.constant(10)
b = tf.constant(20)
c = tf.add(a, b)
# 创建会话对象：
sess = tf.Session()
# 执行操作c
print("不使用上下文管理器：\n", sess.run(c))
# 关闭会话
sess.close()

不使用上下文管理器：
 30


In [10]:
# 2、使用上下文管理器：
a = tf.constant(10)
b = tf.constant(20)
c = tf.add(a, b)
with tf.Session() as sess:
    print("使用上下文管理器：\n", sess.run(c))

使用上下文管理器：
 30


**tf.Session(target='', graph=None, config=None)**

`target`：如果将此参数留空（默认设置），会话将仅使用本地计算机中的设备。
可以指定 `grpc://` 网址，以便指定 TensorFlow 服务器的地址，这使得会话可以访问该服务器控制的计算机上的所有设备。

会话可以分配不同的资源在不同的设备上运行。

```python
/job:worker/replica:0/task:0/device:CPU:0
```


`graph`：默认情况下，新的 `tf.Session` 将绑定到当前的默认图。


`config`：此参数允许您指定一个 `tf.ConfigProto` 以便控制会话的行为。例如，`ConfigProto` 协议用于打印设备使用信息

```python
# 运行会话并打印设备信息
sess = tf.Session(config=tf.ConfigProto(allow_soft_placement=True,
                                        log_device_placement=True))
```

**tf.Session.run()**

`run(fetches, feed_dict=None, options=None, run_metadata=None)`

通过使用 `sess.run()` 来运行 `operation`

`fetches`：单一的 `operation`，或者列表、元组(其它不属于 tensorflow 的类型不行)

`feed_dict`：参数允许调用者覆盖图中张量的值，运行时赋值
> 与 `tf.placeholder` 搭配使用，则会检查值的形状是否与占位符兼容。

使用 `tf.operation.eval()` 也可运行 `operation`，但需要在会话中运行

In [11]:
# 创建图
a = tf.constant(5.0)
b = tf.constant(6.0)
c = a * b

# 创建会话
sess = tf.Session()

# 计算C的值
print(sess.run(c))
print(c.eval(session=sess))

30.0
30.0


`placeholder` 提供占位符，`run` 时候通过 `feed_dict` 指定参数。

In [14]:
# 实例化会话对象
a = tf.constant(10)
b = tf.constant(20)
c = tf.add(a, b)

# 定义 placeholder
ph = tf.placeholder(dtype=tf.int32, shape=[None, 4])

# 会话的创建
# 1. 实例化一个 Session 对象, 运行会话并打印设备信息
sess = tf.Session(
    config=tf.ConfigProto(
      allow_soft_placement=True,
      log_device_placement=True
    )
)

print("a, b, c的值为: %s" % sess.run([a, b, c]))
print("c.eval: %s" % c.eval(session=sess))
a, b, c = sess.run([a, b, c])
print("a: %s" % a)
print("b: %s" % b)
print("c: %s" % c)

# sess 方式运行 placeholder
print("placeholder: %s" % sess.run(
    ph,
    feed_dict={
        ph: [
          [2, 4, 6, 8],
          [1, 3, 5, 7]
        ]
    }
  )
)
# eval 方式运行 placeholder
print("ph_eval: %s" % ph.eval(
    session=sess,
    feed_dict={
        ph: [
          [2, 4, 6, 8],
          [1, 3, 5, 7]
        ]
    }
  )
)
# 2. 用完会话的资源之后应该把资源回收
sess.close()

Device mapping:
/job:localhost/replica:0/task:0/device:XLA_CPU:0 -> device: XLA_CPU device

a, b, c的值为: [10, 20, 30]
c.eval: 30
a: 10
b: 20
c: 30
placeholder: [[2 4 6 8]
 [1 3 5 7]]
ph_eval: [[2 4 6 8]
 [1 3 5 7]]


张量的阶

In [16]:
tensor1 = tf.constant(4.0)
tensor2 = tf.constant([1, 2, 3, 4])
linear_squares = tf.constant([[4], [9], [16], [25]], dtype=tf.int32)

print(tensor1.shape)
print(tensor2.shape)
print(linear_squares.shape)

()
(4,)
(4, 1)


常用的创建张量的指令：

```python
# 创建所有元素设置为 0 的张量
tf.zeros(shape, dtype=tf.float32, name=None)

# 创建所有元素设置为 1 的张量
tf.ones(shape, dtype=tf.float32, name=None)

# 创建一个常数张量
tf.constant(value, dtype=None, shape=None, name="Const")

# 从正态分布中输出随机值, 由随机正态分布的数字组成的矩阵
tf.random_normal(
  shape, mean=0.0, stddev=1.0,
  dtype=float32, seed=None, name=None
)
```

张量类型转换

```python
tf.string_to_number(string_tensor, out_type=None, name=None)
tf.cast(x, dtype, name=None)
```

张量形状改变

- 动态形状: `tf.reshape`
- 静态形状: `tf.set_shape`

> 转换静态形状的时候，`1-D` 到 `1-D`，`2-D` 到 `2-D`，不能跨阶数改变形状，对于已经固定的张量的静态形状的张量，不能再次设置静态形状
> `tf.reshape()` 动态创建新张量时，张量的元素个数必须匹配

In [18]:
# 能用 set_shape 就优先使用，不能用的话，使用 reshape
tensor1 = tf.constant([1, 2, 3, 4])
ph = tf.placeholder(tf.float32, [None, 4])
# tensor2 = tf.constant(11.0)

# tensor1.set_shape([2, 2])  错误
ph.set_shape([3, 4])

new_tensor = tf.reshape(ph, [4, 3])

print(f"ph: {ph}, new_tensor: {new_tensor}")
# 打印的形状,在定义数据的时候形状使用[]
# 不确定的形状:（？， 4）

with tf.Session() as sess:
    print(sess.run(tensor1))

ph: Tensor("Placeholder_4:0", shape=(3, 4), dtype=float32), new_tensor: Tensor("Reshape_1:0", shape=(4, 3), dtype=float32)
[1 2 3 4]
